In [36]:
import streamlit as st
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [37]:
search_for = "Hospital"
search_for = search_for.replace(" ", "+")

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", 
            "Accept-Encoding":"gzip, deflate",
            "Accept-Language":"en,en-IN;q=0.9,en-US;q=0.8,hi;q=0.7",
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
            "DNT":"1",
            "Connection":"close", 
            "Upgrade-Insecure-Requests":"1"} 

In [38]:
latitude, longitude = 12.992650, 77.668480

zoom = 16
# height = 4700

delta_lat = 0.02
delta_long = delta_lat/np.cos(delta_lat*np.pi/180)

latlonglist = [(latitude+i, longitude+j) for i in [k*delta_lat for k in range(-2, 3)] 
                for j in [k*delta_long for k in range(-2, 3)]]

queryData = []

In [39]:
loopcounter = 0
for (lat, long) in latlonglist:
        
        link = ("https://www.google.co.in/maps/search/" + search_for + "/@" + str(lat) + "," + 
                str(long) + "," + str(zoom)+ "z/data=!3m1!4b1!4m4!2m3!5m1!4e3!6e5?hl=en&authuser=0")

        html_text = requests.get(link, headers=headers).text

        startList = [m.start() for m in re.finditer(r"\d+ reviews", html_text)]
        strchunks = [html_text[startList[i]:startList[i+1]] for i in range(0, len(startList)-1)]
        strchunks.append(html_text[startList[-1]:])

        for i in strchunks:
                chunckedlist = i.split(",")
                raters = int(re.findall("\d+", chunckedlist[7])[0])
                if raters < 30:
                        continue
                rating = float(chunckedlist[6])
                if rating < 4.0:
                        continue

                name = ""
                prevname = ""
                namelist = []
                counter = 0
                found = 0
                for i in chunckedlist[:100]:
                        if (i.startswith('\\"0x')) and (found==0):
                                found = 1
                                partCounter = counter
                        if (name=="") and (i.startswith('\\"') and (not (i.endswith('\\"') or i.endswith('\\"]')))):
                                name = i
                        elif (name != "") and (not i.endswith('\\"')):
                                name = name + i
                        elif (name != "") and (i.endswith('\\"')):
                                name = name + i
                                if len(name) >= len(prevname):
                                        prevname = name
                                        name = ""
                        counter += 1
                descrs = prevname.replace("\\", '').replace('"', '')
                loclat = float(re.findall("-?\d+\.?\d*", chunckedlist[partCounter-2])[0])
                loclong = float(re.findall("-?\d+\.?\d*", chunckedlist[partCounter-1])[0])
                if ((abs(abs(loclat)-abs(latitude)) > 0.1) or (abs(abs(loclong)-abs(longitude)) > 0.1)):
                        continue
                queryData.append([descrs, rating, raters, loclat, loclong])
        loopcounter += 1
        print(f"Completed {loopcounter} out of {len(latlonglist)}")

df = pd.DataFrame(queryData, columns=['Descrs', 'Rating', 'Raters', 'Latitude', 'Longitude'])
df = df.drop_duplicates()

df['Scaled Rating'] = df['Rating']*(1 - np.power(1.25, -1*np.sqrt(df['Raters'])))
df = df.sort_values("Scaled Rating", ascending=False)

df["Dist"] = (np.sqrt((((df['Latitude']-latitude)*111.3188)**2) + 
                      (((df['Longitude']-longitude)*np.cos(latitude*0.0174)*111.3188)**2)))

df.to_csv("result.csv", index=False)
midLat = (df['Latitude']*df['Scaled Rating']).mean()/(df['Scaled Rating'].mean())
midLong = (df['Longitude']*df['Scaled Rating']).mean()/(df['Scaled Rating'].mean())
print(midLat, midLong)

Completed 1 out of 25
Completed 2 out of 25
Completed 3 out of 25
Completed 4 out of 25
Completed 5 out of 25
Completed 6 out of 25
Completed 7 out of 25
Completed 8 out of 25
Completed 9 out of 25
Completed 10 out of 25
Completed 11 out of 25
Completed 12 out of 25
Completed 13 out of 25
Completed 14 out of 25
Completed 15 out of 25
Completed 16 out of 25
Completed 17 out of 25
Completed 18 out of 25
Completed 19 out of 25
Completed 20 out of 25
Completed 21 out of 25
Completed 22 out of 25
Completed 23 out of 25
Completed 24 out of 25
Completed 25 out of 25
12.997999034671702 77.6526111810294
